In [17]:
import load
import numpy as np
import matplotlib
matplotlib.use("Qt5Agg")
import matplotlib.pyplot as plt
import tree
#import makegal

In [18]:
from general import defaults
#from galaxymodule import quick_mock
dfl = defaults.Default()
dfl.dir_repo

'/home/hoseung/Work/pyclusterevol/repo/'

In [3]:
from galaxymodule import make_gal

In [19]:
cd ~/Work/data/Horizon-AGN/

/home/hoseung/Work/data/Horizon-AGN


In [21]:
cd ~/Work/data/NH/

/home/hoseung/Work/data/NH


In [22]:
nout = 312#782 # 312
s = load.sim.Sim(nout=nout)
gcat = tree.halomodule.Halo(nout=nout, is_gal=True)

[sim._hilbert_cpulist] No AMR instance,
[sim._hilbert_cpulist] Loading one...
An AMR instance is created
 Sim
An AMR instance is created
 Sim
Simulation set up.
[Halo.load_info] loading info
[Halo.load_info] nout = 312, base =./


In [28]:
gg = load.rd_GM.Gal(nout, catalog=gcat.data[40].copy(), info=s.info)
#gg = load.rd_GM.Gal(nout, catalog=gcat.data[104].copy(), info=s.info)
gg.debug=False
make_gal.mk_gal(gg, mstar_min=1e8)
print(np.log10(gg.meta.mstar))
#gg1.cal_norm_vec()
#gg1.meta.nvec

info <load.info.Info object at 0x7fa7fcc859e8>
File Not Found: ./halo/HAL_00312/hal_dms_0000041
No DM data loaded
File Not Found: ./GalaxyMaker/CELL_00312/gal_cells_0000041
No CELL data loaded
catalog center -0.7064976692199707 1.6153959035873413 0.759282648563385
star x [-0.70816618 -0.71355885 -0.706716   ..., -0.70647836 -0.70663089
 -0.70647675]
star x [-1.66851282 -7.06118345 -0.21833181 ...,  0.0193119  -0.13321638
  0.02092123]
gal.debug False
9.65161492897


In [30]:
plt.hist(gg.star["time"])
plt.show()

In [29]:
plt.hist2d(gg.star["x"], gg.star["y"], weights=13.7-gg.star["time"], bins=200)
plt.show()

In [13]:
from matplotlib.colors import LogNorm

In [14]:
plt.hist2d(gg.star["x"], gg.star["y"], weights=13.7-gg.star["time"],
           bins=200, norm=LogNorm())
plt.show()

In [34]:
import time

In [54]:
arr = np.arange(4)
mat = np.random.rand(4,3)


In [93]:
mat

array([[ 0.83086452,  0.23718903,  0.32862803],
       [ 0.2366768 ,  0.33746605,  0.75426078],
       [ 0.1154754 ,  0.3073384 ,  0.19447749],
       [ 0.74724667,  0.62018871,  0.83527797]])

In [105]:
np.tensordot(mat.T, arr, axes=1)

array([ 2.70936759,  2.81270899,  3.64904966])

In [92]:
np.multiply(mat.T, arr)

array([[ 0.        ,  0.2366768 ,  0.23095079,  2.24174   ],
       [ 0.        ,  0.33746605,  0.6146768 ,  1.86056613],
       [ 0.        ,  0.75426078,  0.38895498,  2.5058339 ]])

In [62]:
def tonumpyarray(mp_arr):
    return np.frombuffer(mp_arr.get_obj())

from multiprocessing import Process, Value, Array
import numpy as np
class Simplemock():
    def __init__(self, repo="/home/hoseung/Work/pyclusterevol/repo/sed/",
                 filter_system="SDSS",
                 sed_model="bc03",
                 load=True, 
                 parallel=True):
        self.filter_system = filter_system
        self.repo = repo
        self.sed_model = sed_model
        self.parallel = parallel
        if load:
            self.load_all()
            
    def load_all(self):
        self.load_filters()
        self.load_SED_wavelength()
        return self.load_SED_all()

    def load_filters(self):
        if self.filter_system == "SDSS":
            filter_lambda, filter_u, filter_g, filter_r, filter_i, filter_z = \
                np.genfromtxt(self.repo + "filter_sdss.dat",
                                skip_header=1, unpack=True)

            self.filters = {"lambda":filter_lambda,
                            "u":filter_u,
                            "g":filter_g,
                            "r":filter_r,
                            "i":filter_i,
                            "z":filter_z}

    def load_SED_wavelength(self):
        if self.sed_model == "bc03":
            self.sed_wavelength = np.genfromtxt(self.repo + "lambda.dat")

    def load_SED_all(self):
        """
        Full SEDs are just a few tens of MB.
        """
        if self.sed_model == "bc03":
            self.metal_points = np.array([0.0004, 0.001, 0.004, 0.01, 0.02, 0.04])
            # age points in tables.
            self.age_points = np.genfromtxt(self.repo+"ages_yybc.dat") # in Gry unit
            
            
            if self.parallel:
                SED_shared=Array('d', np.zeros(6*221*1221))
                self.SEDs = tonumpyarray(SED_shared).reshape(6,221,1221)
                print(self.SEDs.shape)
            else:
                self.SEDs = np.zeros((6, 221, 1221))
                
            
            for i, metal in enumerate(self.metal_points):
                #self.SEDs[i,:,:] = np.genfromtxt(self.repo +
                self.SEDs[i,:,:] = np.genfromtxt(self.repo +
                                            "bc03_yy_{:.4f}".format(metal)).reshape(221, 1221)

            if self.parallel:
                return SED_shared
            #else:
            #    return SEDs
        else:
            print("Sorry, Only bc03 is implemented.")

            
def memory():
    """
    Get node total memory and memory usage
    """
    with open('/proc/meminfo', 'r') as mem:
        ret = {}
        tmp = 0
        for i in mem:
            sline = i.split()
            if str(sline[0]) == 'MemTotal:':
                ret['total'] = int(sline[1])
            elif str(sline[0]) in ('MemFree:', 'Buffers:', 'Cached:'):
                tmp += int(sline[1])
        ret['free'] = tmp
        ret['used'] = int(ret['total']) - int(ret['free'])
    return ret            


def get_flux(extinction = False,
             metal_lower_cut = False,
             filter_name='r'):
    #import gc
    #import os
    ### star data ########################################################
    global star
    global SEDs
    global MockSED
    
    #print("PID ",os.getpid(), "reference counter", len(gc.get_referrers(star)))
    
    starmetal = star["metal"] # Is the original array modified?
    if metal_lower_cut:
        # No star with metallicity lower than the lowest table.
        # Modifying array will result in meory copy!!
        starmetal[starmetal < min(MockSED.metal_points)] = min(MockSED.metal_points) * 1.0001

    locate_metal = np.digitize(starmetal, MockSED.metal_points)-1 # GOOD
    relevant_metals = MockSED.metal_points[:max(locate_metal)+2]
    nmetals = len(relevant_metals)

    # Star Age
    t_univ = 13.7
    starage = t_univ - star["time"]

    locate_age = np.digitize(starage, MockSED.age_points)-1 # GOOD
    relevant_ages = MockSED.age_points[:max(locate_age)+2]
    nages = len(relevant_ages)

    ### Filter optimization. #################################################

    # Pick one
    this_filter = MockSED.filters[filter_name]

    # band range
    i_filter_pos = this_filter > 0

    this_filter = this_filter[i_filter_pos]
    filter_lambda_this_band = MockSED.filters["lambda"][i_filter_pos]

    lambda_min_this_band = min(filter_lambda_this_band)
    lambda_max_this_band = max(filter_lambda_this_band)

    i_lambda_min = np.argmax(MockSED.sed_wavelength > lambda_min_this_band) -1
    #print(i_lambda_min, wavelength[i_lambda_min], lambda_min_this_band)
    i_lambda_max = np.argmax(MockSED.sed_wavelength > lambda_max_this_band)
    #print(i_lambda_max, wavelength[i_lambda_max], lambda_max_this_band)

    # Only a small part of SED is needed.
    wavelength = MockSED.sed_wavelength[i_lambda_min:i_lambda_max+1]
    n_wavelength = i_lambda_max - i_lambda_min + 1

    ##### Caclulate band flux #################
    # Load only necessary data
    # Load all once, keep under the class and copy part of it when needed heere.
    seds = np.zeros((nmetals, nages, n_wavelength)) # metal age lambda
    nages_org =221
    nmetal_org=6
    n_wavs_org=1221
    if MockSED.sed_model == "bc03":
        for i, metal in enumerate(relevant_metals):
            for j, age in enumerate(relevant_ages):
                seds[i,j,:] = SEDs[ i * nages_org*n_wavs_org
                                  + j * n_wavs_org + i_lambda_min:
                                    i * nages_org*n_wavs_org
                                  + j * n_wavs_org + i_lambda_max+1]

    # All are array-wise calculations.
    # interpolation weight
    dmda = np.sqrt((relevant_metals[locate_metal+1] - relevant_metals[locate_metal]) * (relevant_ages[locate_age+1] - relevant_ages[locate_age]))
    dl_m = (starmetal - relevant_metals[locate_metal] )/dmda# / \
           #(relevant_metals[locate_metal+1] - relevant_metals[locate_metal])
    dr_m = (relevant_metals[locate_metal+1] - starmetal)/dmda# / \
           #(relevant_metals[locate_metal+1] - relevant_metals[locate_metal])
    dl_a = (starage - relevant_ages[locate_age] )#   / \
           #(relevant_ages[locate_age+1] - relevant_ages[locate_age])
    dr_a = (relevant_ages[locate_age+1] - starage )# / \
           #(relevant_ages[locate_age+1] - relevant_ages[locate_age])

    dmda=0
    # 2D linear interpolation
    # weight * SED.

    filter_in_sed_wavelengths = np.interp(wavelength, filter_lambda_this_band, this_filter)
    
    # This part consumes too much memory. 
    # Reduce memory usage by...
    # Merging all calculation into one line?
    # 
    ## from here
    
    #
    # Querying the amount of avaiable memory is likely of no use.
    # Reading the memory state from /proc/meminfo is not instantly reflected when
    # multiple cores are competing for memory resource.
    
    Flux = \
    np.multiply(filter_in_sed_wavelengths,
                np.multiply( (dr_m * dr_a), seds[locate_metal, locate_age,:].T).T +\
                np.multiply( (dl_m * dr_a), seds[locate_metal + 1, locate_age,:].T).T +\
                np.multiply( (dr_m * dl_a), seds[locate_metal, locate_age + 1, :].T).T +\
                np.multiply( (dl_m * dl_a), seds[locate_metal + 1, locate_age + 1,:].T).T)
    print(memory())
    #Flux = np.multiply(filter_in_sed_wavelengths, Flux)
    ## to here
    
    # UNIT
    
    
    
    
    if not extinction:
        return np.sum(Flux, axis=1)
    else:
        print("Extinction - Not yet implemented")
        return

    # Convolve filter
    # Wavelengths at which filter function are defined are different from the SED wavelength points.
    # Interpolate filter function on SED points.
    

In [153]:
SED_shared=Array('d', np.zeros((6*221*1221)))#, lock=False)
SEDs = tonumpyarray(SED_shared).reshape(6,221,1221)
print(SEDs.shape)

(6, 221, 1221)


In [47]:
global MockSED
global SEDs
MockSED = Simplemock(repo=dfl.dir_repo+'sed/', load=False)
SEDs = MockSED.load_all()
MockSED.SEDs=0
#self = MockSED#.SEDs
global star
star = gg.star
#gg.star["metal"][gg.star["metal"] >= 0.04] = 0.04 - 1e-9
## Improve Boundary cases. !

(6, 221, 1221)


Even everything to be shared are global, 
having a subprocess referencing to the original memory modifies the original memory.
That's because Python needs to count references to a memory so that it can clean up the memory when no other objects are referencing the memory. 
That means, just having one more subprocess means another copy of memory. 

A great article on this problem. 
https://llvllatrix.wordpress.com/2016/02/19/python-vs-copy-on-write/


#### The core of our solution is to tell the operating system that it should be using shared memory that isn’t reference counted. 

In [58]:
print(len(gc.get_referrers(SEDs)))

1


In [63]:
import multiprocessing as mp
import gc

mp.set_start_method('fork', force=True)

Ps = []
# args must be a tuple. (something,) note the trailing comma.
t0 = time.time()

for filtername in ['u','g','r','i','z']:
    p=mp.Process(target = get_flux, kwargs={"filter_name":filtername})
    p.start()
    #print(len(gc.get_referrers(MockSED)))
    Ps.append(p)
    
for p in Ps:
    p.join()

print(time.time()-t0)

/home/hoseung/anaconda3/envs/hspy/lib/python3.6/site-packages/ipykernel/__main__.py:159: RuntimeWarning: invalid value encountered in sqrt
/home/hoseung/anaconda3/envs/hspy/lib/python3.6/site-packages/ipykernel/__main__.py:159: RuntimeWarning: invalid value encountered in sqrt
/home/hoseung/anaconda3/envs/hspy/lib/python3.6/site-packages/ipykernel/__main__.py:159: RuntimeWarning: invalid value encountered in sqrt
/home/hoseung/anaconda3/envs/hspy/lib/python3.6/site-packages/ipykernel/__main__.py:159: RuntimeWarning: invalid value encountered in sqrt
/home/hoseung/anaconda3/envs/hspy/lib/python3.6/site-packages/ipykernel/__main__.py:159: RuntimeWarning: invalid value encountered in sqrt


{'total': 7896756, 'free': 2515100, 'used': 5381656}
{'total': 7896756, 'free': 3225636, 'used': 4671120}
{'total': 7896756, 'free': 4129060, 'used': 3767696}
{'total': 7896756, 'free': 4275596, 'used': 3621160}
{'total': 7896756, 'free': 5668024, 'used': 2228732}
5.037376642227173


In [51]:
t0 = time.time()
Flux_u = get_flux(filter_name='u')
Flux_g = get_flux(filter_name='g')
Flux_r = get_flux(filter_name='r')
Flux_i = get_flux(filter_name='i')
Flux_z = get_flux(filter_name='z')
print(time.time()-t0)

/home/hoseung/anaconda3/envs/hspy/lib/python3.6/site-packages/ipykernel/__main__.py:140: RuntimeWarning: invalid value encountered in sqrt


(486883,)
(486883,)
(486883,)
(486883,)
(486883,)
0.5114421844482422
